In [2]:

import cv2 as cv
import numpy as np

img1 = np.zeros((11,11))
cv.line(img1, (0,5), (10,5), color=255, thickness=2)
img1

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [255., 255., 255., 255., 255., 255., 255., 255., 255., 255., 255.],
       [255., 255., 255., 255., 255., 255., 255., 255., 255., 255., 255.],
       [255., 255., 255., 255., 255., 255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [3]:
from PIL import Image, ImageDraw

img2 = np.zeros((11,11))
img2 = Image.fromarray(img2).convert("RGB")
img_draw = ImageDraw.Draw(img2)
img_draw.line(((0,5), (10,5)), fill=255, width=3)
img = np.array(img2)[:,:,0]
img

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
      dtype=uint8)

In [55]:

# 看看samRoad和RNGDet的分割结果的统计数据如何

samRoad_mask_path = "/home/godx/research/source_code/sam_road/save/infer__20240427_172057/mask/8_road.png"
RNGDet_mask_paht = "/home/godx/research/RNGDet_test/RNGDetPlusPlus/cityscale/RNGDet_multi_ins/test/segmentation/8_segment.png"

# np.where()
sam_mask = np.array(Image.open(samRoad_mask_path))  # openCV保存的灰度图像，1通道
sam_mask = sam_mask[np.nonzero(sam_mask)]

RNG_mask = np.array(Image.open(RNGDet_mask_paht))[:,:,0]    # PIL保存的灰度图像（转换为了RGB，3通道）
RNG_mask = RNG_mask[RNG_mask.nonzero()]


print(f"SamRoad mean: {np.mean(sam_mask)}, std: {np.std(sam_mask)}")
print(f"RNGDet mean: {np.mean(RNG_mask)}, std: {np.std(RNG_mask)}")


SamRoad mean: 99.67379715348346, std: 102.16144125281158
RNGDet mean: 52.2361188099854, std: 66.21138844370032


In [51]:

# 测试单通道的影像是否可以直接保存
# yes!

single_channel_img = np.ones((256, 256), dtype=np.uint8)*255
print(single_channel_img.shape)
Image.fromarray(single_channel_img).save("./single_cha_img.png")

single_channel_img = np.array(Image.open("./single_cha_img.png"))
print(single_channel_img.shape)

(256, 256)
(256, 256)


In [41]:

# 找到array的非零元素

a = np.array([[1, 2, 0],
              [0, 4, 0],
              [1, 0, 0]])
idxs = np.where(a!=0)   # idxs = ([0, 0, 1, 2],[0, 1, 1, 0]) 手动也行^_^

# 行列数组索引
print(idxs)
print(a[idxs])

# 对应位置布尔类型索引
print(a!=0)
print(a[a!=0])

# 行列数组索引
print(a.nonzero())
print(a[a.nonzero()])

# 行列数组索引
print(np.nonzero(a))
print(a[np.nonzero(a)])

(array([0, 0, 1, 2]), array([0, 1, 1, 0]))
[1 2 4 1]
[[ True  True False]
 [False  True False]
 [ True False False]]
[1 2 4 1]
(array([0, 0, 1, 2]), array([0, 1, 1, 0]))
[1 2 4 1]
(array([0, 0, 1, 2]), array([0, 1, 1, 0]))
[1 2 4 1]
